In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
test = pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')
sub = pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
sub.head()

In [ ]:
all_df = pd.concat([train, test], sort = False)

In [ ]:
all_df

In [ ]:
all_df['year'] = all_df['Dates'].astype('datetime64').dt.year

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# countplot - 
# barplot - 
plt.figure(figsize = (15,10))
sns.barplot(all_df['Category'], all_df['year'])
plt.ylim(2005, 2010) # y limit
plt.xticks(rotation=75) # sns.barplot 밑에 써야함

# 요새 많이 발생하는 범죄, 예전에 발생하던 범죄를 볼 수 있음
# 검은색 선은 편차. 검은색 선이 길면 편차가 심함. 짧으면 저 연도에만 특히 일어났다고 해석하면 됨.


In [ ]:
all_df.head(50)

In [ ]:
# all_df Address에 / 이 들어있냐, Block of 들어있는지 확인하는 피쳐 넣으면 점수 오름

all_df['isBlock'] = all_df['Address'].str.contains('Block', case=False) # case 대소문자 구분 안한다는 옵션(uppercase, lowercase 이런말인듯)

all_df['isAV'] = all_df['Address'].str.contains('AV', case=False) # case 대소문자 구분 안한다는 옵션(uppercase, lowercase 이런말인듯)
all_df['isSla'] = all_df['Address'].str.contains('/', case=False) # case 대소문자 구분 안한다는 옵션(uppercase, lowercase 이런말인듯)
all_df['isStreet'] = all_df['Address'].str.contains('st', case=False) # case 대소문자 구분 안한다는 옵션(uppercase, lowercase 이런말인듯)


# all_df.groupby('Category')['isBlock'].mean()
# barplot, countplot 봐보자!

In [ ]:
from keras.preprocessing.text import Tokenizer

tk = Tokenizer()
tk.fit_on_texts(all_df['Address'])

In [ ]:
all_text = tk.texts_to_sequences(all_df['Address'])

In [ ]:
all_text[0]

In [ ]:
from keras.preprocessing.sequence import pad_sequences

pad_text = pad_sequences(all_text)

In [ ]:
pad_text[0] # pad를 앞에 채우는 것이 뒤에 채우는 것보다 점수가 보통 더 높음. 
# 왜냐 앞쪽에 중요한 말이 안나옴. 보통 뒷쪽에 중요한 말이 나옴. '안녕하세요? 잘 지내시죠? 저는 캐글러입니다.' 이처럼 앞은 인사말처럼 똑같은 말 나옴.
# 캐글러라는 중요한 정보는 뒤에있음.
# 문장이 길면 pad 잘라서 활용해야 함. 여기선 짧으니 그냥 사용

In [ ]:
pad_train = pad_text[:len(train)]
pad_test = pad_text[len(train):]

In [ ]:
tk.word_index # 가장 많이 등장한 순서대로 index 찍힘. isOf, isBlock, isMission, isMarket 이런 피쳐 넣어주면 도움.
# padding 때문에 0 도 있음. Embedding(len(tk.word_index) + 1) // +1 꼭 해주자

In [ ]:
# 아웃풋을 39개로 정해서 모델 만들어서 피쳐로 활용해보자.

In [ ]:
from keras import Sequential
from keras.layers import Embedding, Dense, Flatten

model = Sequential()
model.add(Embedding(len(tk.word_index) + 1, 10, input_length=len(pad_text[0])))
# padding 때문에 0 도 있음. Embedding(len(tk.word_index) + 1) // +1 꼭 해주자, 2번째인자는 임베딩 차원, 3번째 인자는 단어의 길이
model.add(Flatten())
model.add(Dense(39, activation='softmax')) # 정답값 39개

In [ ]:
model.summary()
# flatten_1 (Flatten)          (None, 9)                 0       9*1 해서 9

In [ ]:
model.compile(
    metrics=['acc'], # 대괄호 까먹지 말자
    loss='sparse_categorical_crossentropy', # 머신러닝에서 딥러닝 쓸 때 sparse 씀. 딥러닝은 무조건 숫자여야함. y값을 숫자로 바꿔주기 위해서.
    optimizer='adam'

)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(train['Category'])

In [ ]:
model.fit(pad_train, y, batch_size = 512, epochs=5) # 단어를 학습시키는 것!!

In [ ]:
result1 = model.predict(pad_train)
result2 = model.predict(pad_test)

In [ ]:
result1[0] # 39차원 pca활용해서 

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca.fit(pd.concat([pd.DataFrame(result1), pd.DataFrame(result2)]))

In [ ]:
result_train = pca.transform(result1)
result_test = pca.transform(result2)

In [ ]:
result_train

In [ ]:
all_df

In [ ]:
df_pca = pd.concat([pd.DataFrame(result_train), pd.DataFrame(result_test)])

In [ ]:
all_df = pd.concat([all_df, df_pca], axis = 1)

In [ ]:
all_df

In [ ]:
all_df['month'] = all_df['Dates'].astype('Datetime64').dt.month
all_df['day'] = all_df['Dates'].astype('Datetime64').dt.day
all_df['hour'] = all_df['Dates'].astype('Datetime64').dt.hour
all_df['minute'] = all_df['Dates'].astype('Datetime64').dt.minute


In [ ]:
# 최초 날짜부터 몇일 지났는지 새로운 피쳐 추가하자.
all_df['ndays'] = all_df['Dates'].astype('Datetime64').dt.date - all_df['Dates'].astype('Datetime64').dt.date.min()

In [ ]:
all_df['ndays'] = all_df['ndays'].apply(lambda x: x.days)

In [ ]:
all_df['x_minus_y'] = all_df['X'] - all_df['Y']
all_df['x_plus_y'] = all_df['X'] + all_df['Y']
# 45도 회전변환, 새로운 공간의 좌표를 표시해줄수있음

In [ ]:
all_df.head()

In [ ]:
train

In [ ]:
# minute이 도움이 되는 이유
# boxplot, groupby으로 보기 힘듬, 둘다 카테고리임 minute이랑 category 피쳐 둘다 카테고리
# 이때는, countplot이 좋음
import matplotlib.pyplot as plt
import seaborn as sns
train['minute'] = train['Dates'].astype('Datetime64').dt.minute
plt.figure(figsize=(15,10))
sns.countplot(train['minute'], hue=train['Category']) # 클래스 39개라 그림이 너무 이상해짐, 많은 범죄 5개로 볼수있음.
plt.xlim(-0.5, 1.5)
# 분마다 클래스들의 비율이 달라짐 따라서 도움이 됨.

In [ ]:
train

In [ ]:
all_df = all_df.drop(['Dates', 'Category', 'Descript', 'Id', 'Resolution'], axis = 1)

In [ ]:
# all_df.nunique()
# Address   24777 unique 값이 너무 많음


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

all_df['DayOfWeek'] = le.fit_transform(all_df['DayOfWeek'])
all_df['PdDistrict'] = le.fit_transform(all_df['PdDistrict'])
all_df['Address'] = le.fit_transform(all_df['Address']) # 문장 전처리 방법 알아야한다. 위에서 작업했어도남겨두자

In [ ]:
train2 = all_df[:len(train)] 
test2 = all_df[len(train):]

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train2, train['Category'], test_size=0.2, random_state=33, stratify=train['Category'])

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(num_leaves= 100, learning_rate=0.025, n_estimators=200)
lgb.fit(x_train, y_train, eval_set=(x_valid, y_valid))

# 연도 추가했더니 점수 더 나빠짐 - 왜그럴까? - address 가 매우 중요해서 year 추가해도 점수가 안오르거나 오히려 떨어짐.
# catboost도 사용해서 앙상블쓰자 - 다만 느린게 함정, gpu 쓰면 훨씬 빨라짐.

In [ ]:
from catboost import CatBoostClassifier

cb = CatBoostClassifier(task_type='GPU')
cb.fit(x_train, y_train, eval_set=(x_valid, y_valid), early_stopping_rounds=20)

In [ ]:
# 파라미터 조절 및 앙상블 하자.